In [1]:
R.Version()$version.string

[1] "R version 4.1.0 (2021-05-18)"

In [2]:
library("GENIE3")
library(Seurat)
library(SeuratDisk)

Attaching SeuratObject

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat



In [11]:
set.seed(123)

## Convert H5 Files

In [5]:
Convert("../../nbdata/iPSC_rna_ctr_preprocessed.h5ad", dest = "h5seurat", overwrite = TRUE)
Convert("../../nbdata/iPSC_rna_als_preprocessed.h5ad", dest = "h5seurat", overwrite = TRUE)

Warning message:
“Unknown file type: h5ad”
Warning message:
“'assay' not set, setting to 'RNA'”
Creating h5Seurat file for version 3.1.5.9900

Adding X as scale.data

Adding X as data

Adding X as counts

Adding meta.features from var

Warning message:
“Unknown file type: h5ad”
Warning message:
“'assay' not set, setting to 'RNA'”
Creating h5Seurat file for version 3.1.5.9900

Adding X as scale.data

Adding X as data

Adding X as counts

Adding meta.features from var



## Load Data

In [6]:
rnaCtr <- LoadH5Seurat("../../nbdata/iPSC_rna_ctr_preprocessed.h5seurat")
rnaAls <- LoadH5Seurat("../../nbdata/iPSC_rna_als_preprocessed.h5seurat")

Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding scale.data for RNA

Adding feature-level metadata for RNA

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results

Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding scale.data for RNA

Adding feature-level metadata for RNA

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results



In [7]:
rnaCtrAssay <- GetAssayData(rnaCtr)
rnaAlsAssay <- GetAssayData(rnaAls)

In [10]:
dim(rnaCtrAssay)
dim(rnaAlsAssay)

[1] 29420     8

[1] 32718    12

In [14]:
tfList <- scan("../../nbdata/human_tflist.txt", what="", sep="\n")

## Run GENIE3

In [19]:
?GENIE3

GENIE3 {GENIE3},R Documentation
exprMatrix,"Expression matrix (genes x samples). Every row is a gene, every column is a sample. The expression matrix can also be provided as one of the Bioconductor classes: ExpressionSet: The matrix will be obtained through exprs(exprMatrix) RangedSummarizedExperiment: The matrix will be obtained through assay(exprMatrix), wich will extract the first assay (usually the counts)"
regulators,"Subset of genes used as candidate regulators. Must be either a vector of gene names, e.g. c(""at_12377"", ""at_10912"") or a vector of indices, e.g. c(1,5,6,7). The default value NULL means that all the genes are used as candidate regulators (which is NOT recommended). To provide different regulators for each gene, providethem as named list."
targets,"Subset of genes to which potential regulators will be calculated. Must be either a vector of indices, e.g. c(1,5,6,7), or a vector of gene names, e.g. c(""at_12377"", ""at_10912""). If NULL (default), regulators will be calculated for all genes in the input matrix."
treeMethod,"Tree-based method used. Must be either ""RF"" for Random Forests (default) or ""ET"" for Extra-Trees."
K,"Number of candidate regulators randomly selected at each tree node (for the determination of the best split). Must be either ""sqrt"" for the square root of the total number of candidate regulators (default), ""all"" for the total number of candidate regulators, or a stricly positive integer."
nTrees,Number of trees in an ensemble for each target gene. Default: 1000.
nCores,Number of cores to use for parallel computing. Default: 1.
returnMatrix,Returns output as weight matrix (TRUE). Otherwise (FALSE) it is returned as a list.
verbose,"If set to TRUE, a feedback on the progress of the calculations is given. Default: FALSE."


In [33]:
install.packages('doParallel')
install.packages('doRNG')

also installing the dependency ‘rngtools’




In [34]:
rnaCtrWeights <- GENIE3(
    rnaCtrAssay,
    regulators = intersect(rownames(rnaCtr), tfList),
    verbose = TRUE, 
    nCores = 32
)

Tree method: RF
K: sqrt
Number of trees: 1000


Using 32 cores.



In [41]:
rnaAlsWeights <- GENIE3(
    rnaAlsAssay,
    regulators = intersect(rownames(rnaAls), tfList),
    verbose = TRUE, 
    nCores = 32
)

Tree method: RF
K: sqrt
Number of trees: 1000


Using 32 cores.



In [43]:
write.table(rnaCtrWeights, file = "../../nbdata/iPSC_rna_ctr_genie_weights.csv", sep=",")
write.table(rnaAlsWeights, file = "../../nbdata/iPSC_rna_als_genie_weights.csv", sep=",")